In [29]:
import argparse
import random
from tqdm import tqdm
import os
import sys
os.chdir("..")
#print("\nDIR: ", os.getcwd())
#sys.path.append("C:/Users/Charaf Kamel/Desktop/Kamel/EPFL/Semester_1/Machine_Learning/ML_project/Project_2_swiss_ai_github/ethel-tutor-eval/utils")
%load_ext autoreload
%autoreload 2
parent_dir = os.getcwd()
sys.path.append(parent_dir)
from utils.config import Config
from evaluation.gsm8k_task import GSM8KNShot
from evaluation.base_eval_task import EvalTask
from evaluation.math_task import MATHFewShot
from evaluation.tutoreval_task import TutorEvalTask
from our_datasets.gsm8k_dataset import GSM8K
#from datasets.gsm8k_dataset import GSM8K
from our_datasets.base_dataset import BaseDataset
from our_datasets.math_dataset import MATH
from our_datasets.tutoreval_dataset import TutorEval
from models.ethel import EthelModel
from models.ollama import OllamaModel
from models.smol import SmolModel
#from grader_models.smol_grader import SmolGrader

from utils.recorder import Recorder

random.seed(239)

""" # Parse command-line arguments
parser = argparse.ArgumentParser(description="Run Dataset Evaluation")
parser.add_argument("--dataset", required=True, help="The dataset to use for evaluation: TutorEval")
parser.add_argument("--model", required=True, help="The model to use for evaluation: Ethel, Ollama")
parser.add_argument("--closed_book", required=False, default=True, help="True, if closed_book evaluation is desired, False if the open_book evaluation is desired")
#parser.add_argument("--difficulty", required=False, help="The difficulty level of the question")
 """
os.chdir("C:/Users/Charaf Kamel/Desktop/Kamel/EPFL/Semester_1/Machine_Learning/ML_project/Project_2_swiss_ai_github/ethel-tutor-eval/")
#args = parser.parse_args()
args = argparse.Namespace()
args.dataset="TutorEval"
args.model="Smol"
args.closed_book=True

os.getcwd()
config = Config(
    config_path='config.yaml',
    dataset_dir='data'
)

# Choose dataset class to work with
datasets = {
    'GSM8K': GSM8K,
    'MATH': MATH,
    'TutorEval': TutorEval
}

try:
    dataset_class = datasets[args.dataset]
except KeyError:
    raise ValueError(f"Invalid dataset: {args.dataset}. Supported datasets: {list(datasets.keys())}")

# Define dataset instance and download if necessary
dataset: BaseDataset = dataset_class(config)
dataset.download()

# Load data to the RAM
dataset.load()

# Run Dataset Evaluation
eval_task_class = {
    'GSM8K': GSM8KNShot,
    'MATH': MATHFewShot,
    'TutorEval': TutorEvalTask
}

## TODO: We just care about closed-book evaluation for now

eval_task: EvalTask = eval_task_class[args.dataset](dataset)
print(len(eval_task))
models = {
    'Ethel': EthelModel,
    'Ollama': OllamaModel,
    'Smol': SmolModel
}

try:
    model_class = models[args.model]
except KeyError:
    raise ValueError(f"Invalid model: {args.model}. Supported models: {list(models.keys())}")

model = model_class()

grader_models = {
    'Smol': SmolGrader
}
grader_model = grader_models['Smol']()
recorder = Recorder(config.get_records_path())

i = 0

is_correct_labels = []
print("EVALUATION STARTED")
for ex in tqdm(eval_task, total=len(eval_task)):
    tutor_response = model.generate(ex.messages)
    grader_response = grader_model.feedback(ex, tutor_response)
    generated_answer = eval_task.extract_answer(grader_response)
    

    # Record the iteration data
    recorder.record({
        "input": [m.to_dict() for m in ex.messages],
        "key_points": ex.target,
        "tutor_response": tutor_response,
        "grader_response": grader_response,
        "presentation": generated_answer[0],
        "correctness": generated_answer[1]
    })

    i += 1
    if i == 10:
        break
    #if args.limit is not None and i >= args.limit:
    #    break

print(f"Evaluated model {args.model} on dataset {args.dataset} with {len(is_correct_labels)} examples:")
#print(f"Accuracy: {sum(is_correct_labels) / len(is_correct_labels)}")

recorder.save('evaluation_records.json')

Current working directory: c:\Users\Charaf Kamel\Desktop\Kamel\EPFL\Semester_1\Machine_Learning\ML_project\Project_2_swiss_ai_github\ethel-tutor-eval


INFO:our_datasets.tutoreval_dataset:TutorEval
INFO:our_datasets.tutoreval_dataset:Dataset TutorEval already downloaded


834


NameError: name 'SmolGrader' is not defined

In [ ]:
from datasets import load_dataset
data = load_dataset("princeton-nlp/TutorEval")["train"]

In [ ]:
filtered_dataset = data.filter(lambda x: x['closed_book'] == True)
len(filtered_dataset)


Filter:   0%|          | 0/834 [00:00<?, ? examples/s]

464

In [26]:
out

"Pablo Picasso is considered the father of modern art. He was born in Spain in 1881 and died in 1987. Let's look at his work and his influences. Can you summarize what Pablo Picasso was known for and how he influenced the way artists create?"

In [5]:
generated_answer

[]

In [10]:
eval_task.extract_answer(grader_response)

ANSWER:  The AI's answer is correct. It correctly identifies that radiation is the fastest mode of heat transfer and provides a clear explanation of why. The key points to cover in the question are:

- Radiation is the fastest mode of heat transfer.
- Radiation does not require a medium to travel through.
- Radiation can transfer heat directly from one object to another, regardless of the distance between them.
- Conduction and convection rely on the movement of particles or fluids to transfer heat, which


[0.0, 0.0]

In [43]:
import openai
client = openai.Client(api_key="sk-rc-DYN0lwQUvaV7XKemYdU3zQ", base_url="https://fmapi.swissai.cscs.ch")
res = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "content": "Who is Pablo Picasso?", 
            "role": "user",
        }
    ],
    stream=True,
)
out = ""
for chunk in res:
    if len(chunk.choices) > 0 and chunk.choices[0].delta.content:
        out += chunk.choices[0].delta.content
out

TypeError: Invalid type for url.  Expected str or httpx.URL, got <class 'httpx.URL'>: URL('https://fmapi.swissai.cscs.ch')

In [9]:
from typing import List

import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from utils.custom_stopping import EosListStoppingCriteria
from our_datasets.base_dataset import EvalSample, Message
from models.base_model import BaseModel
from grader_models.base_grader import BaseGrader
import torch
def extract_answer(cls, answer: str) -> str | List[float]:
    print("ANSWER: ", answer)
    try:
        grades = [float(d) for d in re.findall(pattern=r':\s?(\d.*)/3', string=answer)]
    except:
        grades = [0.0, 0.0]
    return grades
import types
eval_task.extract_answer = types.MethodType(extract_answer, eval_task)

In [ ]:
del grader_model
import gc
gc.collect()

7825

In [9]:
tutor_response

Message(role='assistant', content='The fastest mode of heat transfer is radiation. This is because radiation does not require a medium, such as air or water, to travel through. It can transfer heat directly from one object to another, regardless of the distance between them. In contrast, conduction and convection rely on the movement of particles or fluids to transfer heat, which can be slower in certain situations.')

In [3]:
import openai
client = openai.Client(api_key="sk-rc-DYN0lwQUvaV7XKemYdU3zQ", base_url="https://fmapi.swissai.cscs.ch")
res = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "content": "Who is Pablo Picasso?", 
            "role": "user",
        }
    ],
    stream=True,
)


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [5]:
import openai

client = openai.Client(api_key="sk-rc-DYN0lwQUvaV7XKemYdU3zQ", base_url="https://fmapi.swissai.cscs.ch")
res = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "content": "Who is Pablo Picasso?", 
            "role": "user",
        }
    ],
    stream=True,
)

for chunk in res:
    if len(chunk.choices) > 0 and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'